# Artificial and Computational Intelligence Assignment 1

Problem solving by BFS or DFS

Things to follow
1.	Use appropriate data structures to represent the graph and the path using python libraries
2.	Provide proper documentation
3.	Find the path and print it

Coding begins here

1.	Define the robot environment in the following block

In [47]:
#PEAS environment, Initial data structures to define the graph and variable declarations
"""
PEAS Environment

Performance Measure: 
Robot clear all the roads in Chennai for traffic.
Identify shortest route from source city to destination city using BFS and DFS
Show Euler circuit existence and graph
Environment: 
Roads (Connections or edges)
Cities (all city points in the graph)
Algorithms (Euler, BFS and DFS)
Actuators: 
Cities as Node or Vertices
Roads as Connections for Cities
Euler circuit existence check
Selection of source and destination city
Selection of shortest path algorithm either BFS or DFS
Sensors: 
Costs of road
Euler Path existence and Euler Circuit
Shortest path existence comparisons
"""
import tkinter as tk
import tkinter.messagebox as msg
import random
from math import sqrt
from collections import defaultdict
import numpy as np
import time
import csv

bfs_distance = 0
bfs_time = 0
dfs_distance = 0
dfs_time = 0
prev_flag = 0

2.	Define a formula that Checks for existence of Euler path or Euler circuit

In [48]:
"""Is Euler path Exist or not"""

def is_euler_path_exist(gui):
    class Graph:
        """
        This class represents a undirected graph using adjacency list representation
        program to check if a given graph is Eulerian or not and find path
        """

        def __init__(self, vertices):
            self.V = vertices  # No. of vertices
            self.graph = defaultdict(list)  # default dictionary to store graph

        def add_edge(self, vertex_src, vertex_dst):
            """
            function to add an edge to graph
            """
            self.graph[vertex_src].append(vertex_dst)
            self.graph[vertex_dst].append(vertex_src)

        def dfs_util(self, current_vertex, visited):
            """
            A function used by isConnected
            """
            # Mark the current node as visited
            visited[current_vertex] = True

            # Recur for all the vertices adjacent to this vertex
            for connected_vertex in self.graph[current_vertex]:
                if visited[connected_vertex] == False:
                    self.dfs_util(connected_vertex, visited)

        def is_connected(self):
            """
            Method to check if all non-zero degree vertices are
            connected. It mainly does DFS traversal starting from
            node with non-zero degree
            """

            # Mark all the vertices as not visited
            visited = [False] * self.V

            # Find a vertex with non-zero degree
            vertex = 0
            for vertex in range(self.V):
                if len(self.graph[vertex]) > 1:
                    break

            # If there are no edges in the graph, return true
            if vertex == self.V - 1:
                return True

            # Start DFS traversal from a vertex with non-zero degree
            self.dfs_util(vertex, visited)

            # Check if all non-zero degree vertices are visited
            for vertex in range(self.V):
                if visited[vertex] == False and len(self.graph[vertex]) > 0:
                    return False
            return True

        def is_eulerian(self):
            """
            The function returns one of the following values
            0 --> If grpah is not Eulerian
            1 --> If graph has an Euler path (Semi-Eulerian)
            2 --> If graph has an Euler Circuit (Eulerian)
            """
            # Check if all non-zero degree vertices are connected
            if self.is_connected() == False:
                return 0
            else:
                # Count vertices with odd degree
                odd = 0
                for vertex in range(self.V):
                    if len(self.graph[vertex]) % 2 != 0:
                        odd += 1

                '''If odd count is 2, then semi-eulerian. 
                If odd count is 0, then eulerian 
                If count is more than 2, then graph is not Eulerian 
                Note that odd count can never be 1 for undirected graph'''
                if odd == 0:
                    print('GRAPH HAS A EULER CYCLE')
                    return 'GRAPH HAS A EULER CYCLE'
                elif odd == 2:
                    print('GRAPH HAS A EULER PATH')
                    return 'GRAPH HAS A EULER PATH'
                elif odd > 2:
                    pritn('GRAPH IS NOT EULERIAN')
                    return 'GRAPH IS NOT EULERIAN'
    graph = Graph(gui.city_total)
    ovals_dict = {o1['coords']: idx for idx, o1 in enumerate(gui.ovals)}
    for line in gui.lines:
        first = tuple((l1 - 5) for l1 in line['coords'][:2])
        second = tuple((l1 - 5) for l1 in line['coords'][2:])
        vertex_1 = ovals_dict[first]
        vertex_2 = ovals_dict[second]
        graph.add_edge(vertex_1, vertex_2)
    return graph.is_eulerian()

3.	Implementation of BFS for finding the path

In [49]:
""" Function for Best First Search """
def best_first_search(index, start_time):
    stack = []
    found = False
    top = 0
    (x, y) = gui.selected_cities[0]
    (end_x, end_y) = gui.selected_cities[1]

    point = Point(x, y)
    end = Point(end_x, end_y)
    start = point
    point.index = find_in_ovals(x, y)
    stack.append(point)  # source
    visited = np.zeros(gui.city_count)
    visited[point.index] = 1
    popped = 0
    max_element = 0

    while len(stack) > 0 and not found:
        point = stack.pop(top)
        top -= 1
        if point.equal(end):
            found = True
            print("***** PATH FOUND *****")
        else:
            top = add_neighbours(point, stack, top, visited, end, index)
            stack.sort(key=lambda point: point.f, reverse=True)  # sort max to min by h(n)
            if len(stack) > max_element:
                max_element = len(stack)
            popped += 1

    if found:
        total_time = time.time() - start_time
        total_distance, step_size = paint(point, start, max_element, popped)
        return total_distance, step_size, max_element, popped, time.time() - start_time
    else:
        message = "PATH NOT EXISTS"
        gui.step_label.config(text=message)
    return 0, 0, 0, 0, time.time() - start_time

4. Implementation of DFS

In [50]:
""" Function for Depth First Search """
def depth_first_search(start_time):
    stack = []
    found = False
    top = 0
    (x, y) = gui.selected_cities[0]
    (end_x, end_y) = gui.selected_cities[1]

    point = Point(x, y)
    end = Point(end_x, end_y)
    start = point
    point.index = find_in_ovals(x, y)
    stack.append(point)  # source
    visited = np.zeros(gui.city_count)
    visited[point.index] = 1
    popped = 0
    max_element = 0

    while len(stack) > 0 and not found:
        point = stack.pop(top)
        top -= 1
        if point.equal(end):
            found = True
            print("***** PATH FOUND *****")
        else:
            top = add_neighbours(point, stack, top, visited, end, 2)
            if len(stack) > max_element:
                max_element = len(stack)
            popped += 1

    if found:
        total_time = time.time() - start_time
        total_distance, step_size = paint(point, start, max_element, popped)
        return total_distance, step_size, max_element, popped, time.time() - start_time
    else:
        message = "PATH NOT EXISTS"
        gui.step_label.config(text=message)
    return 0, 0, 0, 0, time.time() - start_time

4.	Calling main function

In [51]:
""" Calling BFS and DFS from Run Function """
def run():
    if gui.conn_count < gui.city_total - 1:
        msg.showerror(title="Connection Count Error", message="The number of connections is {} it's less than threshold"
                                                              " {}.".format(gui.conn_count, gui.city_total - 1))
        return
    elif gui.city_count + 1 < gui.city_total:
        msg.showerror(title="City Count Error", message="The number of cities is {} but it's supposed to be {}.".
                      format(gui.city_count, gui.city_total))
        return
    elif gui.conn_count < gui.entry - 1:
        msg.showinfo(title="Connection Count Info", message="The number connections is {} but it's supposed to be {}."
                     .format(gui.conn_count, gui.entry))
    start = time.time()

    if gui.algorithm.get() == 2:  # best first search
        total_distance, step_size, max_element, popped, total_time = best_first_search(1, start)
        message2 = "ALGORITHM : B F S {}\nTOTAL DISTANCE : {}\nTOTAL TIME : {}\nNO OF VERTICES : {" \
                   "}\nCOVERED PATH : {}\nPOPPED : {}" \
            .format('', total_distance, total_time, max_element, step_size, popped)
        gui.output_label.config(text=message2)
        print(message2)
        compare_bfs_dfs_algo(1, total_distance, total_time, message2)

    elif gui.algorithm.get() == 3:
        total_distance, step_size, max_element, popped, total_time = depth_first_search(start)
        message2 = "ALGORITHM : D F S {}\nTOTAL DISTANCE : {}\nTOTAL TIME : {}\nNO OF VERTICES : {" \
                   "}\nCOVERED PATH : {}\nPOPPED : {}" \
            .format('', total_distance, total_time, max_element, step_size, popped)
        gui.output_label.config(text=message2)
        print(message2)
        compare_bfs_dfs_algo(2, total_distance, total_time, message2)
    else:
        msg.showerror(title="Algorithm Selection Error", message="You did not choose the algorithm.")
    return total_distance, step_size, max_element, popped, total_time

5.	The agent should provide the following output

5.1.	Whether an Euler path or circuit exists

In [52]:
def show_euler_path(gui):
    class Graph:
        """
        Program to print Euler Path in a given Cities Graph
        """

        def __init__(self, vertices):
            self.V = vertices  # No. of vertices
            self.graph = defaultdict(list)  # default dictionary to store graph
            self.Time = 0
            self.euler_list = []

        def add_edge(self, vertex_src, vertext_dst):
            """
            function to add an edge to graph
            :param vertex_src:
            :param vertext_dst:
            :return:
            """
            self.graph[vertex_src].append(vertext_dst)
            self.graph[vertext_dst].append(vertex_src)

        def remove_edge(self, vertex_src, vertext_dst):
            """
            This function removes edge vertex_src-vertext_dst from graph
            :param vertex_src:
            :param vertext_dst:
            :return:
            """
            for index, key in enumerate(self.graph[vertex_src]):
                if key == vertext_dst:
                    self.graph[vertex_src].pop(index)
            for index, key in enumerate(self.graph[vertext_dst]):
                if key == vertex_src:
                    self.graph[vertext_dst].pop(index)

        def dfs_count(self, vertex, visited):
            """
            A DFS based function to count reachable vertices from vertex
            :param vertex:
            :param visited:
            :return:
            """
            count = 1
            visited[vertex] = True
            for i in self.graph[vertex]:
                if visited[i] == False:
                    count = count + self.dfs_count(i, visited)
            return count

        def is_valid_next_edge(self, u, v):
            """
            The function to check if edge u-v can be considered as next edge in
            :param u:
            :param v:
            :return:
            """
            # The edge u-v is valid in one of the following two cases:

            # 1) If v is the only adjacent vertex of u
            if len(self.graph[u]) == 1:
                return True
            else:
                ''' 
                2) If there are multiple adjacents, then u-v is not a bridge 
                    Do following steps to check if u-v is a bridge 

                2.a) count of vertices reachable from u'''
                visited = [False] * (self.V)
                count1 = self.dfs_count(u, visited)

                '''2.b) Remove edge (u, v) and after removing the edge, count 
                    vertices reachable from u'''
                self.remove_edge(u, v)
                visited = [False] * (self.V)
                count2 = self.dfs_count(u, visited)

                # 2.c) Add the edge back to the graph
                self.add_edge(u, v)

                # 2.d) If count1 is greater, then edge (u, v) is a bridge
                return False if count1 > count2 else True

        def print_euler_util(self, u):
            """
            Print Euler path starting from vertex u
            :param u:
            :return:
            """
            # Recur for all the vertices adjacent to this vertex
            for v in self.graph[u]:
                # If edge u-v is not removed and it's a a valid next edge
                if self.is_valid_next_edge(u, v):
                    self.remove_edge(u, v)
                    self.print_euler_util(v)
                    self.euler_list.append(u)

        def print_euler_path(self):
            """
            The main function that print Euler Path. It first finds an odd
            degree vertex (if there is any) and then calls print_euler_util()
            to print the path
            :return:
            """
            # Find a vertex with odd degree
            u = 0
            for i in range(self.V):
                if len(self.graph[i]) % 2 != 0:
                    u = i
                    break
            # Print tour starting from odd vertex
            print("\n")
            self.print_euler_util(u)
    graph = Graph(gui.city_total)
    ovals_dict = {o1['coords']: idx for idx, o1 in enumerate(gui.ovals)}
    for line in gui.lines:
        first = tuple((l1 - 5) for l1 in line['coords'][:2])
        second = tuple((l1 - 5) for l1 in line['coords'][2:])
        vertex_1 = ovals_dict[first]
        vertex_2 = ovals_dict[second]
        graph.add_edge(vertex_1, vertex_2)
    graph.print_euler_path()
    cities_path = '->'.join([gui.cities[v] for v in graph.euler_list]) + '->' + gui.cities[graph.euler_list[0]]    
    print('EULER GRAPH : ')
    print(cities_path)
    return cities_path

5.2.	The Euler path that covers all the edges in the graph

In [53]:
def print_euler_util(self, u):
    """
    Print Euler path starting from vertex u
    :param u:
    :return:
    """
    # Recur for all the vertices adjacent to this vertex
    for v in self.graph[u]:
        # If edge u-v is not removed and it's a a valid next edge
        if self.is_valid_next_edge(u, v):
            self.remove_edge(u, v)
            self.print_euler_util(v)

5.3.	Print the total number of vertices (areas) visited by the agent in finding the path

In [54]:
# Execute code to print the number of vertices travelled to cover the path. (BFS and DFS)

In [55]:
class Gui:
    def __init__(self):

        self.window = root
        self.window.title("ASSIGNMENT 1 , GROUP 11")
        self.bg_color = "pink"
        self.h_color = "navy"
        self.s_color = "RoyalBlue2"
        self.c_color = "Magenta2"
        self.window.configure(bg=self.bg_color)
        self.canvas_size = 700  # canvas
        self.canvas = tk.Canvas(root, width=self.canvas_size, height=self.canvas_size, background="white")
        self.canvas.grid(column=1, row=0)
        self.refresh_canvas()

        self.frame = tk.Frame(self.window, bg=self.bg_color)
        self.frame.grid(row=0, column=0, sticky="n")

        self.frame_list = tk.Frame(self.window, bg=self.bg_color)
        self.frame_list.grid(row=0, column=2, sticky="n")

        self.city_total = 6

        self.input_type = tk.StringVar()
        self.city_or_conn = tk.IntVar()
        self.algorithm = tk.IntVar()
        self.is_euler_path_exist = tk.IntVar()
        self.show_euler_path=tk.IntVar()
        self.entry = 9
        self.dst_src = tk.IntVar()
        self.evaluation = tk.IntVar()
        self.change_type = tk.IntVar()

        self.city_count = 0
        self.conn_count = 0
        self.selected_cities = []
        self.ovals = []
        self.lines = []
        self.gonna_move = -1
        self.gonna_move_cons = []
        self.city = {
            "coords": (0, 0),
            "name": "INDIA"
        }

        self.cities = ["T.Nagar", "Anna Nagar", "Central Station", "Guindy", "Tambaram", "Kotturpuram"]
        self.colors = ["dark violet", "DodgerBlue3", "brown", "LightPink4", "green3", "cyan"]

        header = tk.Label(self.frame, text="PATH FINDING AGENT   ", fg=self.h_color, font='orange 12 bold',
                          bg=self.bg_color)
        header.grid(column=0, row=0)
        section = tk.Label(self.frame, text="NUMBER OF CITIES : 6", font='Calibri 12 bold', fg=self.s_color,
                           bg=self.bg_color)
        section.grid(column=0, row=1, sticky='w')
        section1 = tk.Label(self.frame, text="CONNECTIONS        : 9", font='Calibri 12 bold', fg=self.s_color,
                            bg=self.bg_color)
        section1.grid(column=0, row=2, sticky='w')
        section1 = tk.Label(self.frame, text="EULER CHECKS  : ", font='Calibri 12 bold', fg=self.s_color,
                            bg=self.bg_color)
        section1.grid(column=0, row=3, sticky='w')
        eul1 = tk.Radiobutton(self.frame, text='IS EULER PATH EXISTS    ', command=self.check_euler, value=0,
                              bg=self.bg_color).grid(column=0, row=4)
        eul2 = tk.Radiobutton(self.frame, text='SHOW EULER GRAPH      ', command=self.find_euler_path, value=1,
                              bg=self.bg_color).grid(column=0, row=5)
        section1 = tk.Label(self.frame, text="ALGORITHM USED  : ", font='Calibri 12 bold', fg=self.s_color,
                            bg=self.bg_color)
        section1.grid(column=0, row=6, sticky='w')
        alg1 = tk.Radiobutton(self.frame, text='BREADTH FIRST SEARCH', variable=self.algorithm, value=2,
                              bg=self.bg_color).grid(column=0, row=7)
        alg2 = tk.Radiobutton(self.frame, text='DEPTH FIRST SEARCH     ', variable=self.algorithm, value=3,
                              bg=self.bg_color).grid(column=0, row=8)
        # New code end
        section4 = tk.Label(self.frame, text="DISTANCE CALCULATION : ", font='Calibri 12 bold', fg=self.s_color,
                            bg=self.bg_color)
        section4.grid(column=0, row=9, sticky='w')
        euclidean = tk.Radiobutton(self.frame, text='EUCLEDIAN DISTANCE    ', variable=self.evaluation, value=0,
                                   bg=self.bg_color, ). \
            grid(column=0, row=10)
        manhattan = tk.Radiobutton(self.frame, text='MANHATTAN DISTANCE', variable=self.evaluation, value=1,
                                   bg=self.bg_color). \
            grid(column=0, row=11)
        city_button = tk.Button(self.frame, text="SELECT CIIES", bg=self.s_color, fg=self.h_color,
                                font='Calibri 12 bold', command=self.select_cities).grid(column=0, row=26,
                                                                                         sticky='e', padx=2)
        restart_button = tk.Button(self.frame, text="RESTART", bg="firebrick1", fg=self.h_color,
                                   font='Calibri 12 bold', command=self.def_static_map).grid(column=0,
                                                                                             row=26, sticky='w')
        header3 = tk.Label(self.frame, text="\nOUTPUTS/RESULTS :     ", font='Calibri 14 bold', fg="brown",
                           bg=self.bg_color)
        header3.grid(column=0, row=28)

        self.output_label = tk.Label(self.frame, fg=self.h_color, bg=self.bg_color, font='Calibri 12 bold')
        self.output_label.grid(column=0, row=29, sticky='w')

        self.output_label_1 = tk.Label(self.frame, fg=self.h_color, bg=self.bg_color, font='Calibri 12 bold')
        self.output_label_1.grid(column=0, row=30, sticky='w')

        self.step_label = tk.Label(self.window, fg=self.h_color, bg=self.bg_color, font='Calibri 16 bold')
        self.step_label.grid(column=1, row=1)

    def refresh_canvas(self):
        self.cities = ["T.Nagar", "Anna Nagar", "Central Station", "Guindy", "Tambaram", "Kotturpuram"]
        self.canvas.delete("all")
        self.ovals = []
        self.city_count = 0
        self.conn_count = 0
        self.selected_cities = []
        self.lines = []
        self.canvas.create_line(20, 20, 20, 680, fill="gray")
        self.canvas.create_line(20, 20, 680, 20, fill="gray")
        self.canvas.create_line(680, 20, 680, 680, fill="gray")
        self.canvas.create_line(680, 680, 20, 680, fill="gray")

    def load_same(self):
        self.canvas.delete("all")
        self.city_count = 0
        self.conn_count = 0
        self.canvas.create_line(20, 20, 20, 680, fill="gray")
        self.canvas.create_line(20, 20, 680, 20, fill="gray")
        self.canvas.create_line(680, 20, 680, 680, fill="gray")
        self.canvas.create_line(680, 680, 20, 680, fill="gray")
        self.step_label.config(text=" ")
        self.output_label.config(text=" ")
        self.output_label_1.config(text=" ")
        for i in range(len(self.ovals)):
            x, y = self.ovals[i]["coords"]
            self.canvas.create_oval(x, y, x + 15, y + 15, fill="red", outline=self.h_color)
            if self.cities[self.city_count] == "Le_Havre":
                self.canvas.create_text(x, y + 12, fill=self.h_color, font="Calibri 10", text="Le Havre",
                                        tag=self.cities[self.city_count] + "_text")
            elif self.cities[self.city_count] == "Le_Mans":
                self.canvas.create_text(x, y + 12, fill=self.h_color, font="Calibri 10", text="Le Mans",
                                        tag=self.cities[self.city_count] + "_text")
            else:
                self.canvas.create_text(x, y + 12, fill=self.h_color, font="Calibri 10",
                                        text=self.cities[self.city_count],
                                        tag=self.cities[self.city_count] + "_text")
            self.city_count += 1

        for i in range(len(self.lines)):
            x1, y1, x2, y2 = self.lines[i]["coords"]
            j = random.randint(0, len(self.colors) - 1)
            self.canvas.create_line(x1, y1, x2, y2, fill=self.colors[j])
            self.canvas.create_text((x1 + x2) / 2, (y1 + y2) / 2, fill=self.colors[j], font="Calibri 10",
                                    text=str(int(distance(x1, y1, x2, y2, 1))))
            self.conn_count += 1

    def select_cities(self):
        self.load_same()
        self.canvas.unbind("<Button-1>")
        self.canvas.bind("<Button-1>", self.dst_and_src)
        self.selected_cities = []
        msg.showinfo(title="INFO", message="SELECT SOURCE AND DESTINATION CITY")

    def set_connections(self):
        c = self.city_total.get()
        msg.showinfo(title="Number of Connections", message="Number of connections must be between {} and {}."
                                                            "\nPlease enter number of connections in below text box."
                     .format(c - 1, int(c * (c - 1) / 2)))

    def binder(self):
        self.canvas.unbind("<Button-1>")
        self.canvas.bind("<Button-1>", self.create_by_input)

    def find_city(self, x, y):
        i = 0
        found = False
        while i < len(self.ovals) and not found:
            x1, y1 = self.ovals[i]["coords"]
            if x1 - 1 < x < x1 + 11 and y1 - 1 < y < y1 + 11:  # it is a city
                found = True
            else:
                i += 1
        return found, i

    def find_connections(self, index):
        x, y = self.ovals[index]["coords"]
        for i in range(len(self.lines)):
            x1, y1, x2, y2 = self.lines[i]["coords"]

            if distance(x1 - 5, y1 - 5, x, y, 0) == 0 or distance(x2 - 5, y2 - 5, x, y, 0) == 0:
                self.gonna_move_cons.append(i)
                name = "{},{}-{},{}".format(x1, y1, x2, y2)
                self.canvas.delete(name + "_line")
                self.canvas.delete(name + "_text")

    def overlap(self, x, y):
        overlap = False
        i = 0
        while i < len(self.ovals) and not overlap:
            x1, y1 = self.ovals[i]["coords"]
            if distance(x, y, x1, y1, 1) < 30:
                overlap = True
            i += 1

        return overlap

    def create_line(self, x1, y1, fr, x2, y2, to):
        j = random.randint(0, len(self.colors) - 1)
        name = "{},{}-{},{}".format(x1 + 5, y1 + 5, x2 + 5, y2 + 5)
        # print(name)
        self.canvas.create_line(x1 + 5, y1 + 5, x2 + 5, y2 + 5, fill=self.colors[j], tag=name + "_line")
        self.canvas.create_text((x1 + x2 + 10) / 2, (y1 + y2 + 10) / 2, fill=self.colors[j], font="Calibri 10",
                                text=str(int(distance(x1 + 5, y1 + 5, x2 + 5, y2 + 5, 1))), tag=name + "_text")
        line = {"coords": (x1 + 5, y1 + 5, x2 + 5, y2 + 5)}
        self.lines.append(line)

    def create_city(self, x, y):
        self.canvas.create_oval(x, y, x + 15, y + 15, fill="red", outline=self.h_color,
                                tag=self.cities[self.city_count] + "_oval")
        if self.cities[self.city_count] == "":
            self.canvas.create_text(x, y + 12, fill=self.h_color, font="Calibri 10", text="",
                                    tag=self.cities[self.city_count] + "_text")
        elif self.cities[self.city_count] == "":
            self.canvas.create_text(x, y + 12, fill=self.h_color, font="Calibri 10", text="",
                                    tag=self.cities[self.city_count] + "_text")
        else:
            self.canvas.create_text(x, y + 12, fill=self.h_color, font="Calibri 10", text=self.cities[self.city_count],
                                    tag=self.cities[self.city_count] + "_text")
        city = {"coords": (x, y), "name": self.cities[self.city_count]}
        self.ovals.append(city)

    def def_static_map(self):
        self.refresh_canvas()
        x = 0
        y = 0
        i = 0
        # add cities

        if not self.overlap(300, 100):
            self.create_city(300, 100)
            self.city_count += 1

        if not self.overlap(100, 300):
            self.create_city(100, 300)
            self.city_count += 1

        if not self.overlap(500, 300):
            self.create_city(500, 300)
            self.city_count += 1

        if not self.overlap(100, 500):
            self.create_city(100, 500)
            self.city_count += 1

        if not self.overlap(300, 500):
            self.create_city(300, 500)
            self.city_count += 1

        if not self.overlap(500, 500):
            self.create_city(500, 500)
            self.city_count += 1

        to = random.randint(0, len(self.ovals) - 1)
        self.create_line(300, 100, i, 100, 300, to)
        self.conn_count += 1

        to = random.randint(0, len(self.ovals) - 1)
        self.create_line(300, 100, i, 500, 300, to)
        self.conn_count += 1

        to = random.randint(0, len(self.ovals) - 1)
        self.create_line(100, 300, i, 500, 300, to)
        self.conn_count += 1

        to = random.randint(0, len(self.ovals) - 1)
        self.create_line(100, 300, i, 100, 500, to)
        self.conn_count += 1

        to = random.randint(0, len(self.ovals) - 1)
        self.create_line(100, 500, i, 300, 500, to)
        self.conn_count += 1

        to = random.randint(0, len(self.ovals) - 1)
        self.create_line(300, 500, i, 500, 500, to)
        self.conn_count += 1

        to = random.randint(0, len(self.ovals) - 1)
        self.create_line(100, 300, i, 300, 500, to)
        self.conn_count += 1

        to = random.randint(0, len(self.ovals) - 1)
        self.create_line(300, 500, i, 500, 300, to)
        self.conn_count += 1

        to = random.randint(0, len(self.ovals) - 1)
        self.create_line(500, 500, i, 500, 300, to)
        self.conn_count += 1
        gui.output_label.config(text="")

    def check_euler(self):
        message = is_euler_path_exist(self)
        msg.showinfo(title="EULER", message=message)

    def find_euler_path(self):
        euler_graph = show_euler_path(gui)
        msg.showinfo(title="EULER GRAPH : ", message=euler_graph)

    def is_a_city(self, x, y):
        i = 0
        found = False
        x1 = 0
        y1 = 0
        while i < len(self.ovals) and not found:
            x1, y1 = self.ovals[i]["coords"]
            if x1 - 1 < x < x1 + 11 and y1 - 1 < y < y1 + 11:  # it is a city
                found = True
            i += 1
        return found, x1, y1

    def dst_and_src(self, event):
        x = event.x
        y = event.y
        flag, x, y = self.is_a_city(x, y)
        if flag and (len(self.selected_cities) == 0 or len(self.selected_cities) == 1
                     and self.selected_cities[0] != (x, y)):
            self.selected_cities.append((x, y))
            if len(self.selected_cities) == 1:
                msg.showinfo(title="INFO", message=" SOURCE CITY SELECTED")
            if len(self.selected_cities) == 2:
                msg.showinfo(title="INFO", message=" DESTINATION CITY SELECTED")
                run()


class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.f = 0
        self.g = 0
        self.h = 0
        self.index = 0
        self.parent = None

    def equal(self, other):
        if distance(self.x, self.y, other.x, other.y, 0) == 0:
            return True


def distance(x1, y1, x2, y2, want):
    if gui.evaluation == 0 or want == 1:  # euclidean distance
        return sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)
    else:  # manhattan distance
        return abs(x1 - x2) + abs(y1 - y2)


def find_in_ovals(x, y):
    i = 0
    while i < len(gui.ovals):
        (x1, y1) = gui.ovals[i]["coords"]
        if distance(x, y, x1, y1, 0) == 0:
            return i
        i += 1
    return -1


def append_to_stack(x, y, end_point, parent_point, index, stack, top, visited, index2):
    point = Point(x - 5, y - 5)
    point.parent = parent_point
    if index == 0 or index == 1:  # a star and best first search
        point.h = distance(x - 5, y - 5, end_point.x, end_point.y, 0)
        if index == 0:  # a star
            point.g = point.parent.g + distance(point.x, point.y, point.parent.x, point.parent.y, 1)
        point.f = point.g + point.h
    stack.append(point)
    top += 1
    visited[index2] = 1
    return top


def add_neighbours(parent_point, stack, top, visited, end_point, index):
    # find neighbours
    i = 0
    while i < len(gui.lines):
        x1, y1, x2, y2 = gui.lines[i]["coords"]
        index1 = find_in_ovals(x1 - 5, y1 - 5)
        index2 = find_in_ovals(x2 - 5, y2 - 5)

        if distance(x1 - 5, y1 - 5, parent_point.x, parent_point.y, 0) == 0 and visited[index2] == 0:
            # print("Eklenen nokta: ", x2 - 5, y2 - 5)
            top = append_to_stack(x2, y2, end_point, parent_point, index, stack, top, visited, index2)

        elif distance(x2 - 5, y2 - 5, parent_point.x, parent_point.y, 0) == 0 and visited[index1] == 0:
            # print("Eklenen nokta: ", x1 - 5, y1 - 5)
            top = append_to_stack(x1, y1, end_point, parent_point, index, stack, top, visited, index1)

        i += 1

    return top


def paint(point, start, max_element, pops):
    path = []
    step_size = 0
    total_distance = 0
    while not point.equal(start):
        path.append(point)
        total_distance += distance(point.x, point.y, point.parent.x, point.parent.y, 1)
        point = point.parent
        step_size += 1
    message = "Step Size: {}, Distance: {}".format(step_size, int(total_distance))
    message2 = "Maximum Stack Size: {}\nNumber of Pops: {}".format(max_element, pops)
    for i in range(len(path) - 1, -1, -1):
        gui.canvas.create_line(path[i].x + 5, path[i].y + 5, path[i].parent.x + 5, path[i].parent.y + 5,
                               width=3, fill="yellow")
        gui.window.update()
        time.sleep(0.5)
    return total_distance, step_size


def compare_bfs_dfs_algo(ch_flag, distanceVal, timeVal, message):
    global prev_flag
    global bfs_distance
    global bfs_time
    global dfs_distance
    global dfs_time
    if ch_flag == 1:
        prev_flag = ch_flag
        bfs_distance = distanceVal
        bfs_time = timeVal
    if ch_flag == 2:
        dfs_distance = distanceVal
        dfs_time = timeVal

    if prev_flag == 1 and ch_flag == 2:
        print("COMP SPACE : ", bfs_distance - dfs_distance)
        print("COMP TIME : ", bfs_time - dfs_time)
        message = "\nCOMP SPACE : {}\nCOMP TIME : {}".format(bfs_distance - dfs_distance,
                                                             bfs_time - dfs_time)
        gui.output_label_1.config(text=message)


root = tk.Tk()
gui = Gui()
gui.def_static_map()
root.mainloop()